# Recommender System Based on Milennial Preferences in the Real Estate Sector in Mexico's Capital

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt 
%matplotlib inline
#mas las que se me ocurraaaaannnnn


In [2]:
col = pd.read_csv('coloniascdmx.csv')
col.head()

,COLONIA,ENTIDAD,Geo Point,Geo Shape,CVE_ALC,ALCALDIA,CVE_COL,SECC_COM,SECC_PAR
0,LOMAS DE CHAPULTEPEC,9.0,"19.4228411174,-99.2157935754","{""type"": ""Polygon"", ""coordinates"": [[[-99.2201...",16,MIGUEL HIDALGO,16-042,"4924, 4931, 4932, 4935, 4936, 4940, 4987","4923, 4937, 4938, 4939, 4942"
1,LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC),9.0,"19.4106158914,-99.2262487268","{""type"": ""Polygon"", ""coordinates"": [[[-99.2296...",16,MIGUEL HIDALGO,16-044,4963,4964
2,DEL BOSQUE (POLANCO),9.0,"19.4342189235,-99.2094037513","{""type"": ""Polygon"", ""coordinates"": [[[-99.2082...",16,MIGUEL HIDALGO,16-026,NaN,"4918, 4919"
3,PEDREGAL DE SANTA URSULA I,9.0,"19.314862237,-99.1477954505","{""type"": ""Polygon"", ""coordinates"": [[[-99.1458...",3,COYOACAN,03-135,"433, 500, 431, 513, 501","424, 425, 426, 430, 499"
4,AJUSCO I,9.0,"19.324571116,-99.1561602234","{""type"": ""Polygon"", ""coordinates"": [[[-99.1585...",3,COYOACAN,03-128,"376, 377, 378, 379, 404, 493, 498",374


In [3]:
col.drop(columns = ['ENTIDAD','CVE_COL', 'SECC_COM', 'SECC_PAR', 'CVE_ALC'], inplace = True)
col.head()

,COLONIA,Geo Point,Geo Shape,ALCALDIA
0,LOMAS DE CHAPULTEPEC,"19.4228411174,-99.2157935754","{""type"": ""Polygon"", ""coordinates"": [[[-99.2201...",MIGUEL HIDALGO
1,LOMAS DE REFORMA (LOMAS DE CHAPULTEPEC),"19.4106158914,-99.2262487268","{""type"": ""Polygon"", ""coordinates"": [[[-99.2296...",MIGUEL HIDALGO
2,DEL BOSQUE (POLANCO),"19.4342189235,-99.2094037513","{""type"": ""Polygon"", ""coordinates"": [[[-99.2082...",MIGUEL HIDALGO
3,PEDREGAL DE SANTA URSULA I,"19.314862237,-99.1477954505","{""type"": ""Polygon"", ""coordinates"": [[[-99.1458...",COYOACAN
4,AJUSCO I,"19.324571116,-99.1561602234","{""type"": ""Polygon"", ""coordinates"": [[[-99.1585...",COYOACAN


In [10]:
dfcol = col.groupby(['ALCALDIA']).agg(lambda x: ','.join(x)).reset_index(drop = False)
dfcol = dfcol[]


#toronto.groupby(['Postal code','Borough'], sort = False).agg(lambda x: ','.join(x)).reset_index(level=['Postal code','Borough'])
# Replacing '/' with a comma
#toronto['Neighborhood']=toronto['Neighborhood'].str.replace("/", ',' , regex = True)
# Replacing Neighborhood 'Not Assigned' values with the Boroughs, and reseting the index:
#toronto.loc[toronto[toronto['Neighborhood'] == 'Not assigned'].index,'Neighborhood']= toronto[toronto['Neighborhood'] == 'Not assigned']['Borough']
#toronto.reset_index(drop=True, inplace= True)

,ALCALDIA,COLONIA
0,ALVARO OBREGON,"OLIVAR DEL CONDE 2DA SECCION I,OLIVAR DEL COND..."
1,AZCAPOTZALCO,"SAN JUAN TLIHUACA (PBLO),SAN BARTOLO CAHUALTON..."
2,BENITO JUAREZ,"NARVARTE V,PORTALES III,DEL VALLE VII,NAPOLES ..."
3,COYOACAN,"PEDREGAL DE SANTA URSULA I,AJUSCO I,VISTAS DEL..."
4,CUAJIMALPA DE MORELOS,"AHUATENCO,SAN PABLO CHIMALPA (PBLO),CORREDOR S..."
5,CUAUHTEMOC,"TABACALERA,CENTRO VII,GUERRERO I,NONOALCO-TLAT..."
6,GUSTAVO A. MADERO,"CTM EL RISCO (U HAB),EL ARBOLILLO 2 (U HAB),C ..."
7,IZTACALCO,"LA ASUNCION,EX EJIDOS DE LA MAGDALENA MIXIHUCA..."
8,IZTAPALAPA,"DESARROLLO URBANO QUETZALCOATL II,CAROLOS PAC..."
9,LA MAGDALENA CONTRERAS,"INDEPENDENCIA BATAN SUR (U HAB),LA CONCEPCION,..."
